# Imports

In [1]:
from numba import njit, prange
import numpy as np
import pandas as pd
import timeit

# Check CPU info

In [2]:
cpu_info = !lscpu
for inf_item in cpu_info.get_list():
  print(inf_item)

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Address sizes:                   39 bits physical, 48 bits virtual
Byte Order:                      Little Endian
CPU(s):                          8
On-line CPU(s) list:             0-7
Vendor ID:                       GenuineIntel
Model name:                      Intel(R) Core(TM) i7-4790K CPU @ 4.00GHz
CPU family:                      6
Model:                           60
Thread(s) per core:              2
Core(s) per socket:              4
Socket(s):                       1
Stepping:                        3
CPU(s) scaling MHz:              90%
CPU max MHz:                     4700.0000
CPU min MHz:                     800.0000
BogoMIPS:                        8003.44
Flags:                           fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc arch_perfmon pebs bts rep_good nopl x

## Raw data

The raw data will consist of two numpy "Series" (i.e. 1D arrays) of 1 million random numbers selected from a normal distribution

In [3]:
series1 = np.random.randn(1000000).astype(np.float32)
series1

array([ 0.5694377, -2.11488  ,  0.6751292, ...,  0.568219 , -1.5195571,
       -1.0794889], dtype=float32)

In [4]:
series2 = np.random.randn(1000000).astype(np.float32)
series2

array([-1.0263077 , -0.89608634,  2.496726  , ..., -1.2716848 ,
        0.01034788, -0.512277  ], dtype=float32)

## A very simple arithmetic based method

The aim of the following methods is a very simple summation of the two arrays.

As the inputs are numpy arrays, this is an operation that numpy is already heavily optimised for, and extremely quick at executing.

In [5]:
# plain python (numpy)
def sum_nums(a, b):
    return a + b

# the same method decorated to use numba
@njit
def sum_nums_numba(a, b):
    return a + b

## Check

Let's see if the methods return the same output.

In [6]:
test1 = sum_nums(series1, series2)
test1

array([-0.45687002, -3.0109663 ,  3.1718552 , ..., -0.70346576,
       -1.5092093 , -1.5917659 ], dtype=float32)

In [7]:
test2 = sum_nums_numba(series1, series2)
test2

array([-0.45687002, -3.0109663 ,  3.1718552 , ..., -0.70346576,
       -1.5092093 , -1.5917659 ], dtype=float32)

In [8]:
np.array_equal(test1, test2, equal_nan=False)

True

## Timit setup

To ensure that the comparison of execution is consistent and fair (or as fair as possible), the timit module will be used.

This allows the explicit separation of the setup (not timed) and execution (timed) phases of the test.

In [130]:
# setup without the pre-compilation of the numba function
setup = '''
from numba import njit
import numpy as np

series1 = np.random.randn(1000000).astype(np.float32)
series2 = np.random.randn(1000000).astype(np.float32)

def sum_nums(a, b):
    return a + b

@njit
def sum_nums_numba(a, b):
    return a + b
'''

In [131]:
# setup including the pre-compilation of the numba function.
# By runnning the numba function once it is compiled and therefore
# every future execution uses the compiled version
setup_with_comp = '''
from numba import njit
import numpy as np

series1 = np.random.randn(1000000).astype(np.float32)
series2 = np.random.randn(1000000).astype(np.float32)

def sum_nums(a, b):
    return a + b

@njit
def sum_nums_numba(a, b):
    return a + b

sum_nums_numba(series1, series2)
'''

In [132]:
# the timed numpy code
numpy_func = '''
sum_nums(series1, series2)
'''

In [133]:
# the timed numba code
numba_func = '''
sum_nums_numba(series1, series2)
'''

## Define a method to print the exection time in milliseconds

In [134]:
def print_millis(time, iterations):
    time_millis = (time / iterations) * 1000.0
    print(f"Mean execution time: {time_millis:.2f}ms (for {iterations} iteration(s))")

## Run timit for different methods

### A single iternation

In [161]:
iterations = 1

In [167]:
# numpy
print_millis(timeit.timeit(stmt=numpy_func, setup=setup, number=iterations),iterations)

Mean execution time: 0.78ms (for 1 iteration(s))


In [155]:
# numba including timing for the compilation
print_millis(timeit.timeit(stmt=numba_func, setup=setup, number=iterations),iterations)

Mean execution time: 84.10ms (for 1 iteration(s))


In [156]:
# numba not including timing for the compilation
print_millis(timeit.timeit(stmt=numba_func, setup=setup_with_comp, number=iterations),iterations)

Mean execution time: 0.83ms (for 1 iteration(s))


### 10000 iterations

In [157]:
iterations = 10000

In [158]:
# numpy
print_millis(timeit.timeit(stmt=numpy_func, setup=setup, number=iterations),iterations)

Mean execution time: 0.51ms (for 10000 iteration(s))


In [159]:
# numba including timing for the compilation
print_millis(timeit.timeit(stmt=numba_func, setup=setup, number=iterations),iterations)

Mean execution time: 0.54ms (for 10000 iteration(s))


In [160]:
# numba not including timing for the compilation
print_millis(timeit.timeit(stmt=numba_func, setup=setup_with_comp, number=iterations),iterations)

Mean execution time: 0.53ms (for 10000 iteration(s))


## Python loops

Where numba really comes into play is with looping.

The functions below still use numpy arrays and methods, but are part of a looped operation.

There are three funcitons defined below:

1. loop_function: standard python loop using numpy mathematical functions
2. loop_function_numba: same as 1 but with numba applied
3. loop_function_numba_par: same as 2 with parallel processing applied (in this case 8 threads)

In [88]:
def loop_function(a, b):
    c = np.zeros(a.shape, dtype=np.float32)
    for i in np.arange(c.shape[0]):
        if a[i] < b[i]:
            c[i] = 1.0
        else:
            c[i] = 2.0
    return c, a + b

@njit
def loop_function_numba(a, b):
    c = np.zeros(a.shape, dtype=np.float32)
    for i in np.arange(c.shape[0]):
        if a[i] < b[i]:
            c[i] = 1.0
        else:
            c[i] = 2.0
    return c, a + b

@njit(parallel=True)
def loop_function_numba_par(a, b):
    c = np.zeros(a.shape, dtype=np.float32)
    for i in prange(c.shape[0]):
        if a[i] < b[i]:
            c[i] = 1.0
        else:
            c[i] = 2.0
    return c, a + b

### Test the function output

In [89]:
test3 = loop_function(series1, series2)
test3

(array([2., 1., 1., ..., 2., 1., 1.], dtype=float32),
 array([-0.45687002, -3.0109663 ,  3.1718552 , ..., -0.70346576,
        -1.5092093 , -1.5917659 ], dtype=float32))

In [90]:
test4 = loop_function_numba(series1, series2)
test4

(array([2., 1., 1., ..., 2., 1., 1.], dtype=float32),
 array([-0.45687002, -3.0109663 ,  3.1718552 , ..., -0.70346576,
        -1.5092093 , -1.5917659 ], dtype=float32))

In [91]:
test5 = loop_function_numba_par(series1, series2)
test5

(array([2., 1., 1., ..., 2., 1., 1.], dtype=float32),
 array([-0.45687002, -3.0109663 ,  3.1718552 , ..., -0.70346576,
        -1.5092093 , -1.5917659 ], dtype=float32))

In [92]:
np.array_equal(test3, test4, equal_nan=False)

True

In [93]:
np.array_equal(test3, test5, equal_nan=False)

True

### Setup code with and without compilation of the numba functions

In [168]:
setup_loop = '''
from numba import njit, prange
import numpy as np
series1 = np.random.randn(1000000).astype(np.float32)
series2 = np.random.randn(1000000).astype(np.float32)

def loop_function(a, b):
    c = np.zeros(a.shape, dtype=np.float32)
    for i in np.arange(c.shape[0]):
        if a[i] < b[i]:
            c[i] = 1.0
        else:
            c[i] = 2.0
    return c, a + b

@njit
def loop_function_numba(a, b):
    c = np.zeros(a.shape, dtype=np.float32)
    for i in np.arange(c.shape[0]):
        if a[i] < b[i]:
            c[i] = 1.0
        else:
            c[i] = 2.0
    return c, a + b

@njit(parallel=True)
def loop_function_numba_par(a, b):
    c = np.zeros(a.shape, dtype=np.float32)
    for i in prange(c.shape[0]):
        if a[i] < b[i]:
            c[i] = 1.0
        else:
            c[i] = 2.0
    return c, a + b
'''

In [169]:
setup_loop_with_comp = '''
from numba import njit, prange
import numpy as np
series1 = np.random.randn(1000000).astype(np.float32)
series2 = np.random.randn(1000000).astype(np.float32)

@njit
def loop_function_numba(a, b):
    c = np.zeros(a.shape, dtype=np.float32)
    for i in np.arange(c.shape[0]):
        if a[i] < b[i]:
            c[i] = 1.0
        else:
            c[i] = 2.0
    return c, a + b

@njit(parallel=True)
def loop_function_numba_par(a, b):
    c = np.zeros(a.shape, dtype=np.float32)
    for i in prange(c.shape[0]):
        if a[i] < b[i]:
            c[i] = 1.0
        else:
            c[i] = 2.0
    return c, a + b

loop_function_numba(series1, series2)
loop_function_numba_par(series1, series2)
'''

### The timed functions

In [170]:
numpy_func_loop = '''
loop_function(series1, series2)
'''

In [171]:
numba_func_loop = '''
loop_function_numba(series1, series2)
'''

In [172]:
numba_func_loop_par = '''
loop_function_numba_par(series1, series2)
'''

### A single iteration

In [173]:
iterations = 1

#### numpy

In [174]:
# numpy
print_millis(timeit.timeit(stmt=numpy_func_loop, setup=setup_loop, number=iterations),iterations)

Mean execution time: 264.04ms (for 1 iteration(s))


#### numba

In [175]:
# numba including timing for the compilation
print_millis(timeit.timeit(stmt=numba_func_loop, setup=setup_loop, number=iterations),iterations)

Mean execution time: 120.18ms (for 1 iteration(s))


In [176]:
# numba not including timing for the compilation
print_millis(timeit.timeit(stmt=numba_func_loop, setup=setup_loop_with_comp, number=iterations),iterations)

Mean execution time: 2.67ms (for 1 iteration(s))


#### numba (parallelised)

In [177]:
# numba (parallelised) including timing for the compilation
print_millis(timeit.timeit(stmt=numba_func_loop_par, setup=setup_loop, number=iterations),iterations)

Mean execution time: 259.88ms (for 1 iteration(s))


In [178]:
# numba (parallelised) not including timing for the compilation
print_millis(timeit.timeit(stmt=numba_func_loop_par, setup=setup_loop_with_comp, number=iterations),iterations)

Mean execution time: 1.09ms (for 1 iteration(s))


### 100 iterations

In [179]:
iterations = 100

#### numpy

In [180]:
# numpy
print_millis(timeit.timeit(stmt=numpy_func_loop, setup=setup_loop, number=iterations),iterations)

Mean execution time: 267.63ms (for 100 iteration(s))


#### numba

In [181]:
# numba including timing for the compilation
print_millis(timeit.timeit(stmt=numba_func_loop, setup=setup_loop, number=iterations),iterations)

Mean execution time: 3.81ms (for 100 iteration(s))


In [182]:
# numba not including timing for the compilation
print_millis(timeit.timeit(stmt=numba_func_loop, setup=setup_loop_with_comp, number=iterations),iterations)

Mean execution time: 2.52ms (for 100 iteration(s))


#### numba parallelised

In [183]:
# numba (parallelised) including timing for the compilation
print_millis(timeit.timeit(stmt=numba_func_loop_par, setup=setup_loop, number=iterations),iterations)

Mean execution time: 3.56ms (for 100 iteration(s))


In [184]:
# numba (parallelised) not including timing for the compilation
print_millis(timeit.timeit(stmt=numba_func_loop_par, setup=setup_loop_with_comp, number=iterations),iterations)

Mean execution time: 1.02ms (for 100 iteration(s))


# numba fastmath

In [185]:
@njit(parallel=True, fastmath=True)
def loop_function_numba_par_fast(a, b):
    c = np.zeros(a.shape, dtype=np.float32)
    for i in prange(c.shape[0]):
        if a[i] < b[i]:
            c[i] = 1.0
        else:
            c[i] = 2.0
    return c, a + b

In [186]:
test6 = loop_function_numba_par_fast(series1, series2)
test6

(array([2., 1., 1., ..., 2., 1., 1.], dtype=float32),
 array([-0.45687002, -3.0109663 ,  3.1718552 , ..., -0.70346576,
        -1.5092093 , -1.5917659 ], dtype=float32))

In [187]:
setup_loop_with_comp_fast = '''
from numba import njit, prange
import numpy as np
series1 = np.random.randn(1000000).astype(np.float32)
series2 = np.random.randn(1000000).astype(np.float32)

@njit(parallel=True, fastmath=True)
def loop_function_numba_par_fast(a, b):
    c = np.zeros(a.shape, dtype=np.float32)
    for i in prange(c.shape[0]):
        if a[i] < b[i]:
            c[i] = 1.0
        else:
            c[i] = 2.0
    return c, a + b

loop_function_numba_par_fast(series1, series2)
'''

In [188]:
numba_func_loop_par_fast = '''
loop_function_numba_par_fast(series1, series2)
'''

In [189]:
iterations=100

In [190]:
# numba (parallelised) not including timing for the compilation with fastmath
print_millis(timeit.timeit(stmt=numba_func_loop_par_fast, setup=setup_loop_with_comp_fast, number=iterations),iterations)

Mean execution time: 0.89ms (for 100 iteration(s))


# Matrix Manipulation

In [117]:
matrix1 = np.random.randn(100,100).astype(np.float32)
matrix1

array([[ 0.6994453 ,  0.2300376 , -0.8554918 , ...,  1.1232344 ,
        -1.2047416 ,  1.0120332 ],
       [-1.0471845 ,  0.25232664,  0.6871004 , ...,  0.88497835,
        -1.7322128 , -1.6650851 ],
       [-0.2829231 , -1.4381075 ,  1.6043217 , ..., -0.71000993,
        -0.46195322,  0.39009646],
       ...,
       [-1.1141001 , -1.1735047 ,  1.2007486 , ...,  1.5294341 ,
         1.2630433 , -0.34106147],
       [ 0.6928011 , -1.6441373 , -0.8479871 , ...,  1.5839043 ,
        -0.3604045 ,  1.894234  ],
       [-0.44210443, -0.1456523 ,  0.839356  , ...,  1.522994  ,
        -1.4823914 ,  1.1663219 ]], dtype=float32)

In [118]:
matrix2 = np.random.randn(100,100).astype(np.float32)
matrix2

array([[ 2.2313402 ,  0.26028275, -0.12664457, ...,  0.09477077,
        -0.10181196,  0.7260126 ],
       [ 0.47332847, -0.42653593,  0.21119395, ..., -0.71329194,
         0.3489029 , -0.7703111 ],
       [ 0.960177  ,  0.9044148 ,  0.47771898, ...,  0.92847383,
         0.7864348 , -1.1062806 ],
       ...,
       [ 1.414894  , -0.7967963 ,  0.89885014, ..., -1.1275258 ,
         0.68309695,  0.5034767 ],
       [-0.75443333,  0.3018513 , -0.6630143 , ..., -0.8253597 ,
        -1.9101845 , -3.328567  ],
       [ 2.0630724 , -0.950598  , -1.4393753 , ..., -0.21714285,
         0.3809049 , -0.5428667 ]], dtype=float32)

In [119]:
@njit
def matrix_func(mat_a, mat_b):
    a = mat_a.T
    c = np.dot(mat_b,a)
    d = c.reshape(50,50,4)
    e = mat_b.reshape(50,4,50)
    f = d.reshape(200,50)
    g = e.reshape(50,200)
    h = np.dot(f,g)
    i = h.reshape(40000)
    result = 0.0
    for j in np.arange(i.shape[0]):
        result = result + (i[j] - np.sum(i)) / np.sqrt(abs(np.average(i)))
    return result

In [120]:
matrix_func(matrix1,matrix2)

752197253.9857084

In [121]:
setup_loop = '''
from numba import njit, prange
import numpy as np
matrix1 = np.random.randn(100,100).astype(np.float32)
matrix2 = np.random.randn(100,100).astype(np.float32)

@njit
def matrix_func(mat_a, mat_b):
    a = mat_a.T
    c = np.dot(mat_b,a)
    d = c.reshape(50,50,4)
    e = mat_b.reshape(50,4,50)
    f = d.reshape(200,50)
    g = e.reshape(50,200)
    h = np.dot(f,g)
    i = h.reshape(40000)
    result = 0.0
    for j in np.arange(i.shape[0]):
        result = result + (i[j] - np.sum(i)) / np.sqrt(abs(np.average(i)))
    return result

@njit(parallel=True)
def matrix_func_para(mat_a, mat_b):
    a = mat_a.T
    c = np.dot(mat_b,a)
    d = c.reshape(50,50,4)
    e = mat_b.reshape(50,4,50)
    f = d.reshape(200,50)
    g = e.reshape(50,200)
    h = np.dot(f,g)
    i = h.reshape(40000)
    result = 0.0
    for j in prange(i.shape[0]):
        result = result + (i[j] - np.sum(i)) / np.sqrt(abs(np.average(i)))
    return result
'''

In [122]:
setup_loop_comp = '''
from numba import njit, prange
import numpy as np
matrix1 = np.random.randn(100,100).astype(np.float32)
matrix2 = np.random.randn(100,100).astype(np.float32)

@njit
def matrix_func(mat_a, mat_b):
    a = mat_a.T
    c = np.dot(mat_b,a)
    d = c.reshape(50,50,4)
    e = mat_b.reshape(50,4,50)
    f = d.reshape(200,50)
    g = e.reshape(50,200)
    h = np.dot(f,g)
    i = h.reshape(40000)
    result = 0.0
    for j in np.arange(i.shape[0]):
        result = result + (i[j] - np.sum(i)) / np.sqrt(abs(np.average(i)))
    return result

@njit(parallel=True)
def matrix_func_para(mat_a, mat_b):
    a = mat_a.T
    c = np.dot(mat_b,a)
    d = c.reshape(50,50,4)
    e = mat_b.reshape(50,4,50)
    f = d.reshape(200,50)
    g = e.reshape(50,200)
    h = np.dot(f,g)
    i = h.reshape(40000)
    result = 0.0
    for j in prange(i.shape[0]):
        result = result + (i[j] - np.sum(i)) / np.sqrt(abs(np.average(i)))
    return result

matrix_func(matrix1, matrix2)
matrix_func_para(matrix1, matrix2)
'''

In [123]:
matrix_numba = '''
matrix_func(matrix1, matrix2)
'''

In [124]:
matrix_numba_para = '''
matrix_func_para(matrix1, matrix2)
'''

In [125]:
iterations=20

In [126]:
# numba including timing for the compilation
print_millis(timeit.timeit(stmt=matrix_numba, setup=setup_loop, number=iterations),iterations)

Mean execution time: 2069.57ms (for 20 iteration(s))


In [127]:
# numba not including timing for the compilation
print_millis(timeit.timeit(stmt=matrix_numba, setup=setup_loop_comp, number=iterations),iterations)

Mean execution time: 2059.81ms (for 20 iteration(s))


In [128]:
# numba (parallelised) including timing for the compilation
print_millis(timeit.timeit(stmt=matrix_numba_para, setup=setup_loop, number=iterations),iterations)

Mean execution time: 316.76ms (for 20 iteration(s))


In [129]:
# numba (parallelised) not including timing for the compilation
print_millis(timeit.timeit(stmt=matrix_numba_para, setup=setup_loop_comp, number=iterations),iterations)

Mean execution time: 303.46ms (for 20 iteration(s))


# Encountered errors for the article

With the last benchmarks conducted on matrices there were various adjustments that had to be made just to get the function to run when using Numba.

It should be noted that none of the following are a problem when using straight Python/NumPy without Numba.

1. Cannot use reshape with second arguement (i.e. cannot specify the re-order type 'F', 'C' etc.)
2. np.matmul is not supported and therefore you have to use np.dot
3. [Open bug](https://github.com/numba/numba/issues/5433) - using reshape after transpose requires a copy to be taken or it fails.
4. [Open bug](https://github.com/numba/numba/issues/6714) - no support for integer arrays with np.dot